In [37]:
import re
import requests
from requests.auth import HTTPBasicAuth
import csv
import time
from dotenv import find_dotenv, load_dotenv, get_key

Собираем наши секретные данные из .env файла

In [38]:
dotenv_path = find_dotenv()
load_dotenv(dotenv_path)

login = get_key(dotenv_path, ("UBI_LOGIN"))
password = get_key(dotenv_path, ("UBI_PASSWORD"))
user_agent = get_key(dotenv_path, ("USER_AGENT"))
db = get_key(dotenv_path, ("DATABASE"))

Заходим в юбисофт и берём токен первого уровня

In [39]:
def get_ubisoft_token(login, password, user_agent):
    url="https://public-ubiservices.ubi.com/v3/profiles/sessions"
    headers = {
        'Content-Type': "application/json",
        'Ubi-AppId': "86263886-327a-4328-ac69-527f0d20a237",
        'User-Agent': user_agent
    }
    ubi_auth = HTTPBasicAuth(login, password)

    ubi_res = requests.post(url, headers=headers, auth=ubi_auth)
    ubi_res = ubi_res.json()

    # ubisoft token
    return ubi_res['ticket']

ubi_token = get_ubisoft_token(login, password, user_agent)

Получаем нандо токен к LiveServices

In [40]:
def get_nadeo_token(ubi_token):
    url = "https://prod.trackmania.core.nadeo.online/v2/authentication/token/ubiservices"
    headers = {
        'Content-Type': "application/json",
        'Authorization': "ubi_v1 t=" + ubi_token,
        'User-Agent': user_agent
    }
    body = {
        'audience':'NadeoLiveServices'
    }
    res = requests.post(url, headers=headers, json=body)
    res = res.json()

    access_token = res['accessToken']
    
    return(access_token)

nadeo_token = get_nadeo_token(ubi_token)

Функция для парсинга лидерборда. В url указвается код кампании.

In [41]:
def get_leaderboard(nadeo_token, offset):

    url = f"https://live-services.trackmania.nadeo.live/api/token/leaderboard/group/28524e1c-351d-4b1c-b72e-5b1a6ee02cd1/top?length=100&onlyWorld=1&offset={offset}"

    headers = {
        'Authorization': "nadeo_v1 t=" + nadeo_token,
        'User-Agent': user_agent
    }
    res = requests.get(url, headers=headers)
    res = res.json()
    return res["tops"][0]["top"]

Проверка на русские символы, тем самым отбираем только россиян.

In [42]:
def check_russian_characters(string):
    return (bool(re.search('[а-яА-Я]', string)) or bool(re.search('Russia', string)))

Финалити, открываем csv таблицу и записываем в неё данные об игроках.

In [ ]:
place = 0
with open(db, 'a', newline='') as file:
    writer = csv.writer(file)
    for i in range(100):
        for obj in get_leaderboard(nadeo_token, i*100):
            for key, value in obj.items():
                if isinstance(value, str) and check_russian_characters(value):
                    print([obj["accountId"], obj["sp"]])
                    writer.writerow([place, obj["accountId"], obj["sp"]])
                    place+=1
        time.sleep(5)